In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Softmax, Input, BatchNormalization, Dropout

from sklearn.model_selection import train_test_split

import tensorflow as tf
from load_policy import load_policy
import pickle
import numpy as np
import gym

import tf_util


Using TensorFlow backend.


In [4]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

In [5]:
expert_policy_file = "experts/Walker2d-v1.pkl"
env_name = "Walker2d-v1"
rounds = 1000
max_steps = 1000

In [6]:
data = pickle.load(open(expert_policy_file, "rb"))
policy_fn = load_policy(expert_policy_file)

obs (1, 17) (1, 17)


In [7]:
observations = []
actions = []
rewards = []

with tf.Session() as sess:
#     tf_util.initialize()
    env = gym.make(env_name)
    for i in range(rounds):
        obs = env.reset()
        r = 0
        for s in range(max_steps):
            action = policy_fn(obs[None,:])
            observations.append(obs)
            actions.append(action)
            obs, r_, done, _ = env.step(action)
            rewards.append(r_)
            r += r_
            if done:
                break
        if i and i%100 == 0:
            print(i)
            

[2018-06-06 16:42:21,591] Making new env: Walker2d-v1


100
200
300
400
500
600
700
800
900


In [8]:
observations = np.array(observations)
actions = np.vstack(actions)
rewards = np.array(rewards)

obs_train, obs_valid, act_train, act_valid = \
    train_test_split(observations, actions, test_size = 0.2, random_state = 1998)

In [9]:
actions.shape

(998363, 6)

In [10]:
input_dim = observations.shape[-1]
bc = Sequential()
bc.add(Dense(32, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
bc.add(Dense(512, input_dim=input_dim, kernel_initializer='normal', activation='relu'))

# bc.add(Dense(512, kernel_initializer='normal', activation='relu'))
# bc.add(BatchNormalization())
# bc.add(Dropout(0.5))
bc.add(Dense(32, kernel_initializer='normal', activation='relu'))

bc.add(Dense(6, kernel_initializer='normal'))


In [11]:
bc.compile(optimizer='adam',loss='mse', metrics=['accuracy', 'mae'])

In [13]:
bc.fit(obs_train, act_train, validation_data=[obs_valid, act_valid],  epochs=10, batch_size= 512)

Train on 798690 samples, validate on 199673 samples
Epoch 1/10
798690/798690 [==============================] - 3s 4us/step - loss: 0.0022 - acc: 0.9746 - mean_absolute_error: 0.0344 - val_loss: 0.0021 - val_acc: 0.9750 - val_mean_absolute_error: 0.0335
Epoch 2/10
798690/798690 [==============================] - 3s 4us/step - loss: 0.0020 - acc: 0.9753 - mean_absolute_error: 0.0331 - val_loss: 0.0019 - val_acc: 0.9756 - val_mean_absolute_error: 0.0316
Epoch 3/10
798690/798690 [==============================] - 3s 4us/step - loss: 0.0019 - acc: 0.9762 - mean_absolute_error: 0.0322 - val_loss: 0.0027 - val_acc: 0.9727 - val_mean_absolute_error: 0.0395
Epoch 4/10
798690/798690 [==============================] - 3s 4us/step - loss: 0.0018 - acc: 0.9766 - mean_absolute_error: 0.0314 - val_loss: 0.0026 - val_acc: 0.9664 - val_mean_absolute_error: 0.0384
Epoch 5/10
798690/798690 [==============================] - 3s 4us/step - loss: 0.0018 - acc: 0.9769 - mean_absolute_error: 0.0307 - val_los

In [14]:
rewards.sum()/1000

5509.277511763128

In [17]:
o = np.random.random(17)[None, :]
bc.predict(o)

array([[-0.29866973, -0.7448158 , -1.0156887 ,  2.5458589 , -0.6921852 ,
        -0.40345457]], dtype=float32)

In [18]:
with tf.Session():
    print(policy_fn(o))

[[-1.0374253  -1.5427535  -1.2263341   2.555836    0.31547225 -0.62651265]]


In [23]:
m_rewards = []

def run_model(m, m_rounds, max_steps):
    env = gym.make(env_name)
    for i in range(m_rounds):
        obs = env.reset()
        r = 0
        for s in range(max_steps):
            action = m.predict(obs[None,:])
#             observations.append(obs)
#             actions.append(action)
            obs, r_, done, _ = env.step(action)
            m_rewards.append(r_)
#             r += r_
            if done:
                break

In [24]:
mr = 1000
mstep = 1000
run_model(bc, mr, mstep)

[2018-06-06 17:07:12,149] Making new env: Walker2d-v1


In [25]:
sum(m_rewards)/mr

5496.529803095356

In [26]:
len(m_rewards)

998417

In [30]:
2

2